In [1]:
using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [2]:
a⁺, a⁻ = nonsym_bosonictensors(5) # bosonic operators

(TensorMap(ProductSpace(ℂ^6) ← ProductSpace(ℂ^6)):
 0.0 + 0.0im                 0.0 + 0.0im  …  0.0 + 0.0im
 1.0 + 0.0im                 0.0 + 0.0im     0.0 + 0.0im
 0.0 + 0.0im  1.4142135623730951 + 0.0im     0.0 + 0.0im
 0.0 + 0.0im                 0.0 + 0.0im     0.0 + 0.0im
 0.0 + 0.0im                 0.0 + 0.0im     0.0 + 0.0im
 0.0 + 0.0im                 0.0 + 0.0im  …  0.0 + 0.0im
, TensorMap(ProductSpace(ℂ^6) ← ProductSpace(ℂ^6)):
 0.0 - 0.0im  1.0 - 0.0im  …               0.0 - 0.0im
 0.0 - 0.0im  0.0 - 0.0im                  0.0 - 0.0im
 0.0 - 0.0im  0.0 - 0.0im                  0.0 - 0.0im
 0.0 - 0.0im  0.0 - 0.0im                  0.0 - 0.0im
 0.0 - 0.0im  0.0 - 0.0im     2.23606797749979 - 0.0im
 0.0 - 0.0im  0.0 - 0.0im  …               0.0 - 0.0im
)

In [3]:
a⁺ ⊗ a⁻

TensorMap((ℂ^6 ⊗ ℂ^6) ← (ℂ^6 ⊗ ℂ^6)):
[:, :, 1, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 2, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0

In [11]:
function BoseHubbardHamiltonian(eltype=ComplexF64, symmetry=ℤ{1}, lattice=FiniteChain(12), t=0.01, U=2.0, w=0, M=5)

    a⁺, a⁻ = nonsym_bosonictensors(M) # bosonic operators
    n_op = a⁺ * a⁻ # number operator
    
    return @mpoham sum(-t * (a⁺ ⊗ a⁻ + a⁻ ⊗ a⁺){i, j} for (i, j) in nearest_neighbours(lattice))
    # return @mpoham sum(U/2 * n_op{i} * (n_op{i} - id(ℂ^(M+1))) ⊗ id(ℂ^(M+1)) for (i) in vertices(lattice))

end

BoseHubbardHamiltonian (generic function with 8 methods)

In [12]:
H = BoseHubbardHamiltonian()

MPOHamiltonian{ComplexSpace, TensorMap{ComplexSpace, 2, 2, Trivial, Matrix{ComplexF64}, Nothing, Nothing}, ComplexF64}(MPSKit.SparseMPOSlice{ComplexSpace, TensorMap{ComplexSpace, 2, 2, Trivial, Matrix{ComplexF64}, Nothing, Nothing}, ComplexF64}[[TensorMap((ℂ^1 ⊗ ℂ^6) ← (ℂ^6 ⊗ ℂ^1)):
[:, :, 1, 1] =
 1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 2, 1] =
 0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 3, 1] =
 0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 4, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 5, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im

[:, :, 6, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im
 TensorMap((ℂ^1 ⊗ ℂ^6) ← (ℂ^6 ⊗ ℂ^2)):
[:, :, 1, 1] =
 0.0 + 0.0im  7.346623942310585e-20 + 0.0im  …  0.0 + 0.0im  0.0 + 0.0im

[:, :, 2, 1] =
 0.2581988

In [6]:
state = FiniteMPS(rand, ComplexF64, 12, ℂ^(5+1), ℂ^20)

FiniteMPS{TensorMap{ComplexSpace, 2, 1, Trivial, Matrix{ComplexF64}, Nothing, Nothing}, TensorMap{ComplexSpace, 1, 1, Trivial, Matrix{ComplexF64}, Nothing, Nothing}}(Union{Missing, TensorMap{ComplexSpace, 2, 1, Trivial, Matrix{ComplexF64}, Nothing, Nothing}}[TensorMap((ℂ^1 ⊗ ℂ^6) ← ProductSpace(ℂ^6)):
[:, :, 1] =
 0.11216959483178512 + 0.26083679447085284im  …  0.4659957583361181 + 0.21136151923523133im

[:, :, 2] =
 0.15530820365149958 - 0.013472041287507142im  …  0.08911280179416035 + 0.22960482708496335im

[:, :, 3] =
 0.15440560966782757 - 0.36105855513921625im  …  0.10250665726925993 - 0.07191870855972884im

[:, :, 4] =
 0.41405102099241325 + 0.6578844620420097im  …  -0.25053424511811534 - 0.13116208549730077im

[:, :, 5] =
 0.17805217891703076 + 0.1939752327613716im  …  -0.3300075921733567 - 0.24520132372902653im

[:, :, 6] =
 -0.25502146199107073 - 0.04754485558150806im  …  -0.5845793039879537 + 0.2666501726215722im
, TensorMap((ℂ^6 ⊗ ℂ^6) ← ProductSpace(ℂ^20)):
[:, :, 1] =
    

In [13]:
(groundstate,_) = find_groundstate(state, H, DMRG());

┌ Info: Iteraton 0 error 0.007643137394356413
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 1 error 0.001170088938271403
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 2 error 0.0011631840484917918
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 3 error 0.0003192101190996874
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 4 error 0.0001373559210382154
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


...

┌ Info: Iteraton 95 error 5.543480478888725e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 96 error 4.999967018720117e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 97 error 4.5100241935358675e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 98 error 4.068355410623449e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 99 error 3.6701571690799003e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


In [15]:
E0 = sum(expectation_value(groundstate, H))

-0.6448494022556983 + 1.9283252384545352e-17im